In [1]:
import pandas as pd
import numpy as np

In [2]:
def f(rank):
    return 1/np.sqrt(rank)

In [3]:
def calcul_classement(df, f, calc_sets=False):
    
    n_joueurs = len(df.Joueurs.unique())
    
    ### Résultat de chaque match
    résultats = []
    for match in df.Match.unique():
        scores = list(df[df["Match"]==match]["Scores"])
        
        résultat = -1
        if scores[0] > scores[1]:
            résultat = 1
        elif scores[0] == scores[1]:
            résultat = 0
            
        résultats.append(résultat)
        résultats.append(-résultat)

    df["Résultat"] = résultats

    ### Points de chaque match
    points = []
    for match in df.Match.unique():
        classement = list(df[df["Match"]==match]["Classement"])        
        sets = list(df[df["Match"]==match]["Scores"])
        résultat = list(df[df["Match"]==match]["Résultat"])
        for i in range(2):
            if calc_sets==False:
                point = max(3*np.sqrt(n_joueurs)*résultat[i]*f(classement[1-i]), 0)
                points.append(point) 
            else:
                point = sets[i]*f(classement[1-i]) + (résultat[i]+1)
                points.append(point)
                
        
    df["Points"] = points
    
    ### Calcul des stats de chaque joueur
    df_rang = pd.DataFrame(columns={"Joueur"})
    
    df_rang["Joueur"] = df.Joueurs.unique()
    joués, victoires, nuls, défaites, taux, points_joueurs = [], [], [], [], [], []
    for joueur in df_rang["Joueur"]:
        nb_joués = df[df["Joueurs"]==joueur].shape[0]
        joués.append(nb_joués)
        nb_victoires = list(df[df["Joueurs"]==joueur]["Résultat"]).count(1)
        victoires.append(nb_victoires)
        nb_nuls = list(df[df["Joueurs"]==joueur]["Résultat"]).count(0)
        nuls.append(nb_nuls)
        nb_défaites = list(df[df["Joueurs"]==joueur]["Résultat"]).count(-1)
        défaites.append(nb_défaites)
        taux.append("{}%".format(int(nb_victoires/nb_joués*100)))
        points_joueurs.append(round(sum(list(df[df["Joueurs"]==joueur]["Points"])), 2))

    df_rang["Joués"] = joués
    df_rang["Victoires"] = victoires
    df_rang["Nuls"] = nuls
    df_rang["Défaites"] = défaites
    df_rang["Taux de succès"] = taux
    df_rang["Points"] = points_joueurs

    df_rang.sort_values(by=["Points", "Taux de succès"], ascending=[False, False], inplace=True)
    
    df_rang["Rang"] = range(1, n_joueurs+1)
    
    return df_rang

In [4]:
def classement_squash(filename, f, calc_sets):
    
    df_matchs = pd.read_csv(filename, sep=';', decimal=',')
    
    if df_matchs.Classement.isna().any():
                
        df_previous_matchs = df_matchs.dropna().copy()

        df_previous_rank = calcul_classement(df_previous_matchs, f, calc_sets)

        # Remplissage avec les nouveaux classements
        for i in range(df_matchs.shape[0]):
            if np.isnan(df_matchs["Classement"][i]):
                joueur = df_matchs["Joueurs"][i]
                df_matchs["Classement"][i] = df_previous_rank[df_previous_rank["Joueur"]==joueur]["Rang"]

        # Export de l'historique mis à jour
        df_matchs.to_csv(filename, sep=';', decimal=',', columns=["Match", "Joueurs", "Classement", "Scores"], index=False)
    
    # Classement final
    df_rank = calcul_classement(df_matchs, f, calc_sets)
    
    return df_rank

In [6]:
rank = classement_squash("Historique.csv", f, True)
print(rank.to_string(index=False))

  Joueur  Joués  Victoires  Nuls  Défaites Taux de succès  Points  Rang
     KRa      9          8     0         1            88%   26.63     1
   Colas      3          3     0         0           100%   10.39     2
      LL      5          2     0         3            40%    7.71     3
   Marin      5          1     0         4            20%    4.50     4
  Mateos      6          1     0         5            16%    4.13     5
 Gessien      2          1     0         1            50%    3.13     6
  Sutter      2          0     0         2             0%    0.82     7


In [7]:
df_attribution_points = pd.DataFrame(columns={"Joueur", "Points à gagner (sur un set)"})
df_attribution_points["Joueur"] = rank.Joueur
n_joueurs = len(list(df_attribution_points["Joueur"]))
df_attribution_points["Points à gagner (sur un set)"] = [f(i) for i in range(1, n_joueurs + 1)]
df_attribution_points["Points à gagner (sur un match)"] = df_attribution_points["Points à gagner (sur un set)"]*3 + 2

print(df_attribution_points.to_string(index=False))

  Joueur  Points à gagner (sur un set)  Points à gagner (sur un match)
     KRa                      1.000000                        5.000000
   Colas                      0.707107                        4.121320
      LL                      0.577350                        3.732051
   Marin                      0.500000                        3.500000
  Mateos                      0.447214                        3.341641
 Gessien                      0.408248                        3.224745
  Sutter                      0.377964                        3.133893
